# Overview

StackOverflow is a collaboratively edited question-and-answer site generally
focused on programming topics. Because of the variety of features tracked,
including a variety of feedback metrics, it allows for some open-ended analysis
of user behavior on the site.

The data is available on [S3](s3://thedataincubator-course/spark-stack-data/)
There are three subfolders: allUsers, allPosts, and allVotes which contain
chunked and gzipped xml with the following format:

```xml
<row Body="&lt;p&gt;I always validate my web pages, and I recommend you do the same BUT many large company websites DO NOT and cannot validate because the importance of the website looking exactly the same on all systems requires rules to be broken. &lt;/p&gt;&#10;&#10;&lt;p&gt;In general, valid websites help your page look good even on odd configurations (like cell phones) so you should always at least try to make it validate.&lt;/p&gt;&#10;" CommentCount="0" CreationDate="2008-10-12T20:26:29.397" Id="195995" LastActivityDate="2008-10-12T20:26:29.397" OwnerDisplayName="Eric Wendelin" OwnerUserId="25066" ParentId="195973" PostTypeId="2" Score="0" />
```

A full schema can be found
[here](https://ia801500.us.archive.org/8/items/stackexchange/readme.txt) which
originates from [this](https://archive.org/details/stackexchange).
## Spark Overflow

We'll use some open-source code (you'll need to clone [this
repo](https://github.com/stevenrskelton/SparkOverflow)) to handle the
integration of the StackOverflow xml into Scala.

The scala files in the SparkOverflow project will parse the XML for you,
allowing you to pull out tags by writing `x.creationDate`. Look through
the scala parsing source code (eg. StackTable.scala, Post.scala) to see how
this works. You may also want to look
[here](http://stevenskelton.ca/files/2013/12/Real-Time-Data-Mining-With-Spark.scala)
and [here](http://stevenskelton.ca/real-time-data-mining-spark/) to get a
better grasp of what this code does.

Also noted that add the following line to the build.sbt.

libraryDependencies += "org.apache.spark" %% "spark-core" % "1.2.0"


## upvote_percentage_by_favorites

Each post on StackExchange can be upvoted, downvoted, and favorited. One
"sanity check" we can do is to look at the ratio of upvotes to downvotes as a
function of how many times the post has been favorited.  Using post favorite
counts as the keys for your mapper, calculate the average percentage of upvotes
(upvotes / (upvotes + downvotes)) for the first 50 keys (starting from the
least favorited posts).
  
Do the analysis on the stats.stackexchange.com dataset.

## user_answer_percentage_by_reputation:

Investigate the correlation between a user's reputation and the kind of posts
they make. For the 99 users with the highest reputation, single out posts which
are either questions or answers and look at the percentage of their posts that
are answers (answers / (answers + questions)).

## user_reputation_by_tenure:
If we use the total number of posts made on the site as a metric for tenure, we
can look at the differences between "younger" and "older" users. You can
imagine there might be many interesting features - for now just return the top
100 post counts and the average reputation for every user who has that count.

## quick_answers_by_hour
How long do you have to wait to get your question answered? Look at the set of
ACCEPTED answers which are posted less than three hours after question
creation. What is the average number of these "quick answers" as a function of
the hour of day the question was asked?  You should normalize by how many total
accepted answers are garnered by questions posted in a given hour, just like
we're counting how many quick accepted answers are garnered by questions posted
in a given hour, eg. (quick accepted answers when question hour is 15 / total
accepted answers when question hour is 15).

## identify_veterans_from_first_post_stats
It can be interesting to think about what factors influence a user to remain
active on the site over a long period of time.  In order not to bias the
results towards older users, we'll define a time window between 100 and 150
days after account creation. If the user has made a post in this time, we'll
consider them active and well on their way to being veterans of the site; if
not, they are inactive and were likely brief users.


Let's see if there are differences between the first ever question posts of
"veterans" vs. "brief users". For each group separately, average the score,
views, number of answers, and number of favorites of users' first question.



In [ ]:
package ca.stevenskelton.sparkoverflow

import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import scala.Predef._
import org.apache.spark.rdd.RDD
import org.apache.log4j.{ LogManager, Level }
import org.apache.spark.SparkConf
import scala.math
    

object Main extends App {
 //def main(args: Array[String]){
  import java.io.File

  //val sparkHome = "C:/Program Files/Hadoop/spark-0.8.0-incubating/"

  val inputDir = args(0)
  val outputDir = args(1)
  val minSplits = 4

  //System.getenv("SPARK_HOME"),Seq(System.getenv("SPARK_EXAMPLES_JAR")))
  //LogManager.getRootLogger().setLevel(Level.WARN)
  //System.setProperty("spark.worker.memory", "3g")
  System.setProperty("spark.executor.memory", "5g")
  System.setProperty("spark.rdd.compress", "true")
  //if (!true) {
    //System.setProperty("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    //System.setProperty("spark.kryo.registrator", "ca.stevenskelton.sparkoverflow.KyroRegistrator")
  //}
  
  println("Start spark.")
  val conf = new SparkConf().setAppName("Main")
  conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
        .set("spark.default.parallelism", "12")
        .set("spark.hadoop.validateOutputSpecs", "false")
  conf.registerKryoClasses(Array(classOf[Post], classOf[User], classOf[Vote]))
  val sc = new SparkContext(conf)

  println("Load data")
  //LOAD DATA USING SPARK
  val jsonData = sc.textFile(Post.file.getAbsolutePath, minSplits)
  val objData = jsonData.flatMap(Post.parse)
  objData.cache

  //val posts = objData.keyBy(_.id)

  val jsonVoteData = sc.textFile(Vote.file.getAbsolutePath, minSplits)
  val voteData = jsonVoteData.flatMap(Vote.parse)
  voteData.cache

  val jsonUserData = sc.textFile(User.file.getAbsolutePath, minSplits)
  val userData = jsonUserData.flatMap(User.parse)
  userData.cache


//=====================================*upvote_percentage_by_favorites*===================================================// 
      //val votes = voteData.map(item=>(item.voteTypeId,1)).reduceByKey(_ + _).sortByKey()    //votes is a list      
      val votegroup_fav = voteData.map(item=>((item.postId,item.voteTypeId),1)).reduceByKey((a,b)=>a+b).filter(item=>(item._1._2==5)).map(item=>(item._1._1,item._2))
      val votegroup_down = voteData.map(item=>((item.postId,item.voteTypeId),1)).reduceByKey((a,b)=>a+b).filter(item=>(item._1._2==3)).map(item=>(item._1._1,item._2))
      val votegroup_up = voteData.map(item=>((item.postId,item.voteTypeId),1)).reduceByKey((a,b)=>a+b).filter(item=>(item._1._2==2)).map(item=>(item._1._1,item._2))  
  val join_group_1 = votegroup_fav.fullOuterJoin(votegroup_up).mapValues(x=>(x._1.getOrElse(0),x._2.getOrElse(0)))
  val join_group_2 = join_group_1.fullOuterJoin(votegroup_down).mapValues(x=>(x._1.getOrElse((0,0)),x._2.getOrElse(0)))
  join_group_2.take(50).foreach(println)
  val fav_downup = join_group_2.map(item=>(item._2._1._1,item._2._1._2*1.0/(item._2._1._2+item._2._2))).sortByKey(false)
  println(fav_downup.mapValues(item=>(item,1)).reduceByKey((x, y) => (x._1 + y._1, x._2 + y._2)).mapValues(item=>item._1*1.0/item._2).sortByKey().take(50).mkString(","))*/

  
      
//=====================================*user_answer_percentage_by_reputation*===================================================// 
            
      val user_rep = userData.map(x=> (x.id,x.reputation))
      val post_answer = objData.map(x=>((x.ownerUserId,x.postTypeId),1)).reduceByKey((a,b)=>a+b).filter(x=>(x._1._2==2)).map(x=>(x._1._1,x._2))
      val post_question = objData.map(x=>((x.ownerUserId,x.postTypeId),1)).reduceByKey((a,b)=>a+b).filter(x=>(x._1._2==1)).map(x=>(x._1._1,x._2))
      
      val group1 = user_rep.fullOuterJoin(post_answer).mapValues(x=>(x._1.getOrElse(0),x._2.getOrElse(0)))
      val group2 = group1.fullOuterJoin(post_question).mapValues(x=>(x._1.getOrElse((0,0)),x._2.getOrElse(0))).map(x=>(x._2._1._1,(x._1,x._2._1._2,x._2._2)))
      
      val rep_percent = group2.map(x=>(x._1,(x._2._1,x._2._2*1.0/(x._2._2+x._2._3)))).sortByKey(false).take(99).map(x=>(x._2._1,x._2._2))
           
      println(rep_percent.mkString(","))
      
      
      
      //=====================================*user_reputation_by_tenure*===================================================// 
      
      val user_rep = userData.map(x=> (x.id,x.reputation))
      val post_count = objData.map(x=>(x.ownerUserId,1)).reduceByKey((a,b)=>a+b)
      val rep_count = user_rep.join(post_count).map(x=>(x._2._2,x._2._1)).sortByKey(false).mapValues(item=>(item,1)).reduceByKey((x, y) => (x._1 + y._1, x._2 + y._2)).mapValues(item=>item._1*1.0/item._2)
      println(rep_count.sortByKey(false).take(100).mkString(","))
      
      
      
      //=====================================*quick_answers_by_hour*===================================================// 
            
      val aaRdd = objData.filter(x=>x.postTypeId == 1).map(x=>(x.acceptedAnswerId,(x.creationhour,x.creationDate)))
      val creationdateRdd = objData.map(x=>(x.id,x.creationDate))
      val joingroup = aaRdd.join(creationdateRdd).map(x=>(x._2._1._1,(x._2._1._2,x._2._2)))
      
      val quick_answer = joingroup.filter(x=>(x._2._2-x._2._1)<=3600000*3).map(x=>(x._1,1)).reduceByKey(_+_)
      val total_answer = joingroup.map(x=>(x._1,1)).reduceByKey(_+_)
      val final_group = quick_answer.join(total_answer).map(x=>(x._1,x._2._1*1.0/x._2._2)).sortByKey()
      println(final_group.collect().mkString(","))      
      

      //=====================================*identify_veterans_from_first_post_stats*===================================================//      
      val user_create_rdd = userData.map(x=>(x.id,x.creationDate))
      val post_create_rdd = objData.map(x=>(x.ownerUserId,x.creationDate)).sortBy(_._2)//.reduceByKey((x,y)=>math.min(x,y))
      val joingroup_veteran = user_create_rdd.join(post_create_rdd).map(x=>(x._1,x._2._2-x._2._1)).filter(x=>x._2<=100*24*3600000L || x._2>=150*24*3600000L).reduceByKey((x,y)=>(x)).map(x=>(x._1,1))   // find the veteran users
        
      val user_first_question = objData.map(x=>(x.ownerUserId,(x.postTypeId,x.creationDate))).filter(x=>(x._2._1==1)).map(x=>(x._1,x._2._2)).reduceByKey((x,y)=>math.min(x,y))  // find the first question for the every user
      val joingroup_firstquestion = joingroup_veteran.join(user_first_question).map(x=>((x._2._2,x._1),1))   // find the first question for veteran users ( first_question_creationdate,veteran_userid)
      val post_attribute_rdd = objData.map(x=>((x.creationDate,x.ownerUserId),(x.viewCount,x.score,x.favoriteCount,x.answerCount)))
      
      val result_part1 = joingroup_firstquestion.join(post_attribute_rdd).map(x=>(1,(x._2._2._1 , x._2._2._2 , x._2._2._3 , x._2._2._4 , 1))).reduceByKey((x, y) => (x._1 + y._1, x._2 + y._2,x._3+y._3,x._4+y._4,x._5+y._5)).mapValues(x=>(x._1*1.0/x._5,x._2*1.0/x._5,x._3*1.0/x._5,x._4*1.0/x._5))   
      println(result_part1.collect().mkString(""))
      
        
 //}
}